In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
!pip install -qU langchain_openai langchain_huggingface langchain_core==0.2.38 langchain langchain_community langchain-text-splitters pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 9.6 MB/s eta 0:00:00


In [5]:
!pip install -qU faiss-cpu unstructured==0.15.7 python-pptx==1.0.2 nltk==3.9.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 32.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.5 MB/s eta 0:00:00


In [6]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")

Enter Your OpenAI API Key: ··········


In [7]:
pdf_paths = ["AI_Risk_Management_Framework.pdf",
"Blueprint-for-an-AI-Bill-of-Rights.pdf"]

In [9]:
from langchain_community.document_loaders import PyPDFLoader

pdf_documents = []
for pdf_path in pdf_paths:
    loader = PyPDFLoader(pdf_path)
    pdf_documents.extend(loader.load())

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 750,
    chunk_overlap  = 20,
    length_function = len
)

In [11]:
training_documents = text_splitter.split_documents(pdf_documents)

In [12]:
len(training_documents)

640

In [13]:
import uuid

id_set = set()

for document in training_documents:
  id = str(uuid.uuid4())
  while id in id_set:
    id = uuid.uuid4()
  id_set.add(id)
  document.metadata["id"] = id

In [14]:
training_split_documents = training_documents[:300]
val_split_documents = training_documents[300:350]
test_split_documents = training_documents[350:400]

In [15]:
from langchain_openai import ChatOpenAI

qa_chat_model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

In [16]:
from langchain_core.prompts import ChatPromptTemplate

qa_prompt = """\
Given the following context, you must generate questions based on only the provided context.

You are to generate {n_questions} questions which should be provided in the following format:

1. QUESTION #1
2. QUESTION #2
...

Context:
{context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)

In [17]:
question_generation_chain = qa_prompt_template | qa_chat_model

In [18]:
response = question_generation_chain.invoke({"context": "regulation", "n_questions":2})

In [19]:
response

AIMessage(content='1. What is the purpose of regulation in various industries?  \n2. How do regulations impact businesses and consumers?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 57, 'total_tokens': 80, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_71fab63729', 'finish_reason': 'stop', 'logprobs': None}, id='run-42261cd9-dae6-4062-ba45-7c01a26ba144-0', usage_metadata={'input_tokens': 57, 'output_tokens': 23, 'total_tokens': 80})

In [20]:
import tqdm
def create_questions(documents, n_questions):
  questions = {}
  relevant_docs = {}
  for document in tqdm.tqdm(documents):
    document_content = {"context" : document.page_content, "questions" : []}
    questions_generated = question_generation_chain.invoke({"context": document.page_content, "n_questions": n_questions})
    for question in questions_generated.content.split("\n"):
      question_id = str(uuid.uuid4())
      questions[question_id] = "".join(question.split(".")[1:]).strip()
      relevant_docs[question_id] = [document.metadata["id"]]
  return questions, relevant_docs

We'll use the function to generate training, validation, and test data with `n_questions=2` for each.

In [21]:
training_questions, training_relevant_contexts = create_questions(training_split_documents,2)

100%|██████████| 300/300 [05:40<00:00,  1.13s/it]


In [22]:
val_questions, val_relevant_contexts = create_questions(val_split_documents,2)

100%|██████████| 50/50 [00:44<00:00,  1.11it/s]


In [23]:
test_questions, test_relevant_contexts = create_questions(test_split_documents,2)

100%|██████████| 50/50 [00:49<00:00,  1.01it/s]


In [24]:
import json

training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in training_split_documents}

train_dataset = {
    "questions" : training_questions,
    "relevant_contexts" : training_relevant_contexts,
    "corpus" : training_corpus
}

with open("training_dataset.jsonl", "w") as f:
  json.dump(train_dataset, f)

In [25]:
val_corpus = {val_item.metadata["id"] : val_item.page_content for val_item in val_split_documents}

val_dataset = {
    "questions" : val_questions,
    "relevant_contexts" : val_relevant_contexts,
    "corpus" : val_corpus
}

with open("val_dataset.jsonl", "w") as f:
  json.dump(val_dataset, f)

In [26]:
train_corpus = {test_item.metadata["id"] : test_item.page_content for test_item in test_split_documents}

test_dataset = {
    "questions" : test_questions,
    "relevant_contexts" : test_relevant_contexts,
    "corpus" : train_corpus
}

with open("test_dataset.jsonl", "w") as f:
  json.dump(test_dataset, f)

In [27]:
!pip uninstall -y pyarrow
!pip install -qU sentence_transformers datasets pyarrow==14.0.1

Found existing installation: pyarrow 14.0.2
Uninstalling pyarrow-14.0.2:
  Successfully uninstalled pyarrow-14.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2024.3.1 which is incompatible.


In [28]:
from sentence_transformers import SentenceTransformer

model_id = "Snowflake/snowflake-arctic-embed-m"
model = SentenceTransformer(model_id)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/84.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [29]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

In [30]:
BATCH_SIZE = 20

In [31]:
corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

In [32]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

In [33]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

In [34]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

In [35]:
EPOCHS = 5

In [36]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='midterm-finetuned_arctic',
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50,
)

Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100,Dot Accuracy@1,Dot Accuracy@3,Dot Accuracy@5,Dot Accuracy@10,Dot Precision@1,Dot Precision@3,Dot Precision@5,Dot Precision@10,Dot Recall@1,Dot Recall@3,Dot Recall@5,Dot Recall@10,Dot Ndcg@10,Dot Mrr@10,Dot Map@100
30,No log,No log,0.780000,0.960000,0.980000,1.000000,0.780000,0.320000,0.196000,0.100000,0.780000,0.960000,0.980000,1.000000,0.904303,0.872179,0.872179,0.780000,0.960000,0.980000,1.000000,0.780000,0.320000,0.196000,0.100000,0.780000,0.960000,0.980000,1.000000,0.904303,0.872179,0.872179
50,No log,No log,0.780000,0.990000,0.990000,1.000000,0.780000,0.330000,0.198000,0.100000,0.780000,0.990000,0.990000,1.000000,0.912129,0.881667,0.881667,0.780000,0.990000,0.990000,1.000000,0.780000,0.330000,0.198000,0.100000,0.780000,0.990000,0.990000,1.000000,0.912129,0.881667,0.881667
60,No log,No log,0.790000,0.990000,0.990000,1.000000,0.790000,0.330000,0.198000,0.100000,0.790000,0.990000,0.990000,1.000000,0.915820,0.886667,0.886667,0.790000,0.990000,0.990000,1.000000,0.790000,0.330000,0.198000,0.100000,0.790000,0.990000,0.990000,1.000000,0.915820,0.886667,0.886667
90,No log,No log,0.790000,0.990000,0.990000,1.000000,0.790000,0.330000,0.198000,0.100000,0.790000,0.990000,0.990000,1.000000,0.915820,0.886667,0.886667,0.790000,0.990000,0.990000,1.000000,0.790000,0.330000,0.198000,0.100000,0.790000,0.990000,0.990000,1.000000,0.915820,0.886667,0.886667
100,No log,No log,0.800000,0.990000,0.990000,1.000000,0.800000,0.330000,0.198000,0.100000,0.800000,0.990000,0.990000,1.000000,0.919511,0.891667,0.891667,0.800000,0.990000,0.990000,1.000000,0.800000,0.330000,0.198000,0.100000,0.800000,0.990000,0.990000,1.000000,0.919511,0.891667,0.891667
120,No log,No log,0.800000,0.990000,0.990000,1.000000,0.800000,0.330000,0.198000,0.100000,0.800000,0.990000,0.990000,1.000000,0.919511,0.891667,0.891667,0.800000,0.990000,0.990000,1.000000,0.800000,0.330000,0.198000,0.100000,0.800000,0.990000,0.990000,1.000000,0.919511,0.891667,0.891667
150,No log,No log,0.800000,0.990000,0.990000,1.000000,0.800000,0.330000,0.198000,0.100000,0.800000,0.990000,0.990000,1.000000,0.918202,0.890000,0.890000,0.800000,0.990000,0.990000,1.000000,0.800000,0.330000,0.198000,0.100000,0.800000,0.990000,0.990000,1.000000,0.918202,0.890000,0.890000


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [37]:
from huggingface_hub import notebook_login

notebook_login()

In [38]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('midterm-finetuned_arctic')

model.push_to_hub("midterm-finetuned-arctic")

Some weights of BertModel were not initialized from the model checkpoint at midterm-finetuned_arctic and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

'https://huggingface.co/XicoC/midterm-finetuned-arctic/commit/9d4fa34226adf1b69f4d4dbcb53a6e2d6e66da93'